In [1]:
import rul_datasets

import pandas as pd
import torch

In [2]:
reader = rul_datasets.CmapssReader(fd=1)

In [3]:
"""The features are a list of [numpy arrays][numpy.ndarray] where each array has a shape of
`[num_windows, window_size, num_channels]`:"""
dev_features, dev_targets = reader.load_split("dev")
val_features, val_targets = reader.load_split("val")
test_features, test_targets = reader.load_split("test")

In [4]:
print(f'dev features: {dev_features[0].shape}')
print(f'val features: {val_features[0].shape}')
print(f'test features: {test_features[0].shape}')

dev features: (163, 30, 14)
val features: (211, 30, 14)
test features: (1, 30, 14)


In [5]:
df_dev_targets = pd.DataFrame(dev_targets[0])
df_val_targets = pd.DataFrame(val_targets[0])
df_test_targets = pd.DataFrame(test_targets[0])

In [7]:
dm = rul_datasets.BaselineDataModule(rul_datasets.RulDataModule(reader, batch_size=32))
dm.prepare_data()
dm.setup()

In [8]:
print(dm.data_module.data['dev'][0].shape)
print(dm.data_module.data['val'][0].shape)
print(dm.data_module.data['test'][0].shape)

torch.Size([13818, 14, 30])
torch.Size([3913, 14, 30])
torch.Size([100, 14, 30])


In [12]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        # CNN feature extraction part
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size)

    def forward(self, x):
        x = self.conv(x)

In [13]:
model = CNN(2, 3, 4)

In [21]:
from torchsummary import summary

summary(model)

Layer (type:depth-idx)                   Param #
├─Conv1d: 1-1                            27
Total params: 27
Trainable params: 27
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv1d: 1-1                            27
Total params: 27
Trainable params: 27
Non-trainable params: 0

In [14]:
x = torch.rand(2, 3, 4)

In [15]:
x

tensor([[[0.2545, 0.6554, 0.5211, 0.1695],
         [0.7850, 0.0489, 0.3894, 0.1397],
         [0.2293, 0.5988, 0.0992, 0.4118]],

        [[0.6987, 0.2107, 0.8606, 0.7833],
         [0.3835, 0.7670, 0.0424, 0.7736],
         [0.7820, 0.8198, 0.6475, 0.8035]]])

In [16]:
model.forward(x)

RuntimeError: Given groups=1, weight of size [3, 2, 4], expected input[2, 3, 4] to have 2 channels, but got 3 channels instead